In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px #is a special tool that helps us draw cool, interactive charts.
import mysql.connector

In [ ]:
df = pd.read_csv(r"C:\Users\Python Class\Agridata_Project2\ICRISAT-District Level Data - ICRISAT-District Level Data.csv")
df.head(5)

In [ ]:
df.info()

In [ ]:
#DATA CLEANING - SANITY CHECK OF DATA

# Convert area from 1000 ha to ha
area_columns = [
    'RICE AREA (1000 ha)', 
    'WHEAT AREA (1000 ha)',
    'KHARIF SORGHUM AREA (1000 ha)',
    'RABI SORGHUM AREA (1000 ha)',
    'SORGHUM AREA (1000 ha)',
    'PEARL MILLET AREA (1000 ha)',
    'MAIZE AREA (1000 ha)',
    'FINGER MILLET AREA (1000 ha)',
    'BARLEY AREA (1000 ha)',
    'CHICKPEA AREA (1000 ha)',
    'PIGEONPEA AREA (1000 ha)',
    'MINOR PULSES AREA (1000 ha)',
    'GROUNDNUT AREA (1000 ha)',
    'SESAMUM AREA (1000 ha)',
    'RAPESEED AND MUSTARD AREA (1000 ha)',
    'SAFFLOWER AREA (1000 ha)',
    'CASTOR AREA (1000 ha)',
    'LINSEED AREA (1000 ha)',
    'SUNFLOWER AREA (1000 ha)',
    'SOYABEAN AREA (1000 ha)',
    'OILSEEDS AREA (1000 ha)',
    'SUGARCANE AREA (1000 ha)',
    'COTTON AREA (1000 ha)',
    'FRUITS AREA (1000 ha)', 
    'VEGETABLES AREA (1000 ha)', 
    'FRUITS AND VEGETABLES AREA (1000 ha)',
    'POTATOES AREA (1000 ha)', 
    'ONION AREA (1000 ha)', 
    'FODDER AREA (1000 ha)'
]

# Check which area columns exist in the dataset
area_columns = list(set(df.columns) & set(area_columns))
# Convert area from 1000 ha to ha
df[area_columns] = df[area_columns].apply(pd.to_numeric, errors='coerce') * 1000
# Rename columns dynamically
# df.columns = df.columns.str.replace(r' \(1000 ha\)', '', regex=True)

# Convert production from 1000 tons to kg
production_columns = [
    'RICE PRODUCTION (1000 tons)', 
    'WHEAT PRODUCTION (1000 tons)', 
    'KHARIF SORGHUM PRODUCTION (1000 tons)',  
    'RABI SORGHUM PRODUCTION (1000 tons)',
    'SORGHUM PRODUCTION (1000 tons)',
    'PEARL MILLET PRODUCTION (1000 tons)',
    'MAIZE PRODUCTION (1000 tons)',
    'FINGER MILLET PRODUCTION (1000 tons)',
    'BARLEY PRODUCTION (1000 tons)',
    'CHICKPEA PRODUCTION (1000 tons)',
    'PIGEONPEA PRODUCTION (1000 tons)',
    'MINOR PULSES PRODUCTION (1000 tons)',
    'GROUNDNUT PRODUCTION (1000 tons)',
    'SESAMUM PRODUCTION (1000 tons)',
    'RAPESEED AND MUSTARD PRODUCTION (1000 tons)',
    'SAFFLOWER PRODUCTION (1000 tons)',
    'CASTOR PRODUCTION (1000 tons)',
    'LINSEED PRODUCTION (1000 tons)',
    'SUNFLOWER PRODUCTION (1000 tons)',
    'SOYABEAN PRODUCTION (1000 tons)',
    'OILSEEDS PRODUCTION (1000 tons)',
    'SUGARCANE PRODUCTION (1000 tons)',
    'COTTON PRODUCTION (1000 tons)'
]

# Check which production columns exist in the dataset
production_columns = list(set(df.columns) & set(production_columns))
# Convert production from 1000 tons to kg
df[production_columns] = df[production_columns].apply(pd.to_numeric, errors='coerce') * 1_000_000

#RENAMING THE COLUMNS
df.columns = df.columns.str.replace(r' \(Kg per ha\)| \(1000 ha\)| \(1000 tons\)', '', regex=True)

# Fill missing numerical values with column mean
df.fillna(df.mean(numeric_only=True), inplace=True)

df.columns = df.columns.str.lower()  #changed to lower case
df.columns = df.columns.str.replace(" ", "_")  #replace space with _

# Save cleaned data
df.to_csv("cleaned_agriculture_data.csv", index=False)

print("Data cleaning complete. Saved as 'cleaned_agriculture_data.csv'.")

In [ ]:
#EDA - EXPLORATORY DATA ANALYSIS - process of analyzing and summarizing datasets to understand
# descriptive statistics
df.describe()

In [ ]:
# 1. Top 7 RICE PRODUCTION State Data(Bar_plot)

# Aggregating rice production at the state level
state_rice_production = df.groupby("state_name")["rice_production"].sum().reset_index() 
#We group the data by state,add rice grown per state.If state has data for multiple years, we combine them into one total number.
# reset_index() - without this after grouping state becomes index not columns , to make it has a we use this 

# Select the top 7 states
top_rice_states = state_rice_production.nlargest(7, "rice_production")  #We sort the states by how much rice they grow. AND PICK TOP 7

#  interactive bar plot
fig = px.bar(
    top_rice_states, 
    x="rice_production", 
    y="state_name", 
    text="rice_production",  # Show values on bars
    orientation="h",  #bars go sideways (horizontal)
    title="Top 7 Rice Producing States",
    labels={"rice_production": "Total Rice Production (kg)", "state_name": "State"},
    color="state_name",  # Color by value
    color_discrete_sequence=px.colors.sequential.Blues_r  # Color gradient
)

# Show values when hovering
fig.update_traces(texttemplate="%{text:.2s}", textposition="outside", hoverinfo="x+y")
#texttemplate="%{text:.2s}" → Formatting the Numbers,%{text} → rice production numbers, .2s → formats number in a short form (scientific notation).
#textposition="outside" → Showing Numbers Outside the Bars , hoverinfo="x+y" → What Info to Show When Hovering

# Show the interactive plot
fig.show()

In [ ]:
#2. Top 5 Wheat Producing States Data(Bar_chart)and its percentage(%)(Pie_chart)
state_wheat_production = df.groupby("state_name")["wheat_production"].sum().reset_index()
top_wheat_states = state_wheat_production.nlargest(5,"wheat_production") 

# Create Interactive Bar Chart
fig = px.bar(
    top_wheat_states, 
    x="wheat_production", 
    y="state_name", 
    text="wheat_production",  # Show values on bars
    labels={"wheat_production": "Total Wheat Production", "state_name": "State"},
    title="Top 5 Wheat Producing States",
    color="state_name",  # Different colors for each state
    orientation="h",  # Horizontal bar chart
    color_discrete_sequence=px.colors.sequential.Viridis   # Use Blues_r color palette
)

# Show values on hover
fig.update_traces(texttemplate="%{text:.2s}", textposition="outside", hoverinfo="x+y")

# Show figure
fig.show()

# Calculate Percentage
top_wheat_states["percentage"] = (top_wheat_states["wheat_production"] / top_wheat_states["wheat_production"].sum()) * 100

# Create Interactive Pie Chart
fig = px.pie(
    top_wheat_states, 
    values="wheat_production", 
    names="state_name", 
    title="PERCENTAGE OF Top 5 WHEAT PRODUCING STATES",
    hover_data=["percentage"],  # Show percentage on hover
    labels={"percentage": "Percentage (%)"},
    color="state_name",
    color_discrete_sequence=px.colors.qualitative.Pastel1 ,
    custom_data=["percentage"]
)

# Format hover template
# fig.update_traces(hovertemplate='%{label}: %{value} Tons (%{customdata[0]:.2f}%)')
fig.update_traces(hovertemplate='%{label}')

# Show figure
fig.show()


In [ ]:
#3. Oil seed production by top 5 states
state_oilSeed_production = df.groupby("state_name")["oilseeds_production"].sum().reset_index()
top_oilseed_states = state_oilSeed_production.nlargest(5,"oilseeds_production") 

# Create Interactive Bar Chart
fig = px.bar(
    top_oilseed_states, 
    x="oilseeds_production", 
    y="state_name", 
    text="oilseeds_production",  # Show values on bars
    labels={"oilseeds_production": "Total Oil Seed Production", "state_name": "State"},
    title="Top 5 Oil Seed Producing States",
    color="state_name",  # Different colors for each state
    orientation="h",  # Horizontal bar chart
    color_discrete_sequence=px.colors.sequential.Blues_r   # Use Blues_r color palette
)

# Show values on hover
fig.update_traces(texttemplate="%{text:.2s}", textposition="outside", hoverinfo="x+y")

# Show figure
fig.show()

In [ ]:
#4.Top 7 SUNFLOWER PRODUCTION  State
state_sunflower_production = df.groupby("state_name")["sunflower_production"].sum().reset_index()
top_sunflower_states = state_sunflower_production.nlargest(7,"sunflower_production") 

# Create Interactive Bar Chart
fig = px.bar(
    top_sunflower_states, 
    x="sunflower_production", 
    y="state_name", 
    text="sunflower_production",  # Show values on bars
    labels={"sunflower_production": "Total Sunflower Production", "state_name": "State"},
    title="Top 7 Sunflower Producing States",
    color="state_name",  # Different colors for each state
    orientation="h",  # Horizontal bar chart
    color_discrete_sequence=px.colors.sequential.Viridis   # Use Blues_r color palette
)

# Show values on hover
fig.update_traces(texttemplate="%{text:.2s}", textposition="outside", hoverinfo="x+y")

# Show figure
fig.show()

In [ ]:
#5.India's SUGARCANE PRODUCTION From Last 50 Years(Line_plot)
yearwise_sugarcane_production = df.groupby("year")["sugarcane_production"].sum().reset_index()

sugarcane_prod = yearwise_sugarcane_production.tail(50)

# Create Interactive Line Plot
fig = px.line(
    sugarcane_prod, 
    x="year", 
    y="sugarcane_production", 
    markers=True,  # Show markers at data points
    labels={"sugarcane_production": "Total Sugarcane Production", "year": "Year"},
    title="India's Sugarcane Production Over the Last 50 Years",
    color_discrete_sequence=px.colors.sequential.Viridis   # Use Viridis color palette
)

# Improve hover details
fig.update_traces(mode="lines+markers", hoverinfo="x+y")
#"lines" → Draws a continuous line connecting data points."markers" → Adds distinct points (dots) at each data value for better visibility.
#"lines+markers" → Combines both, ensuring the trend is visible while making individual data points easy to spot.

# Show figure
fig.show()

In [ ]:
#6.Rice Production Vs Wheat Production (Last 50y)
# Assuming df has 'year', 'rice_production', 'wheat_production'
last_50_years = df.tail(50)  

# Melt the data for Plotly
#melt is a function used to unpivot a DataFrame from wide format to long format.
#It helps transform columns into rows, making the data more suitable for analysis and visualization.
melted_df = last_50_years.melt(id_vars=["year"], # Keeps the year column unchanged.
                               value_vars=["rice_production", "wheat_production"], #Specifies which columns to melt.
                               var_name="Crop", #The new column that stores the original column names (rice_production and wheat_production).
                               value_name="Production") #The new column that

fig = px.bar(melted_df, x="year", 
             y="Production", 
             color="Crop",
             barmode="stack",  # Stacked bars
             labels={"Production": "Total Production (in tons)", "year": "Year"},
             title="Rice vs Wheat Production - Last 50 Years in Stacked Bar",
             color_discrete_map={"rice_production": "gold", "wheat_production": "black"})

fig.update_layout(xaxis_tickangle=-45)
fig.show()

In [ ]:
#7.Rice Production By West Bengal Districts
dis_riceProd_WB = df[df["state_name"] == "West Bengal"].groupby("dist_name")["rice_production"].sum().reset_index()

# Create Interactive Bar Chart
fig = px.bar(
    dis_riceProd_WB, 
    x="rice_production", 
    y="dist_name", 
    text="rice_production",  # Show values on bars
    labels={"rice_production": "Total Rice Production", "dist_name": "District"},
    title="RICE PRODUCTION BY WEST BENGAL DISTRICT WISE",
    color="dist_name",  # Different colors for each dis
    orientation="h",  # Horizontal bar chart
    color_discrete_sequence=px.colors.sequential.Blues_r  
)

# Show values on hover
fig.update_traces(texttemplate="%{text:.2s}", textposition="outside", hoverinfo="x+y")

# Show figure
fig.show()

In [ ]:
#8.Top 10 Wheat Production Years From UP
yearwise_wheatProd_UP = df[df["state_name"] == "Uttar Pradesh"].groupby("year")["wheat_production"].sum().reset_index()

top_wheat_yearwise = yearwise_wheatProd_UP.nlargest(10,"wheat_production") 

# Create Interactive Bar Chart
fig = px.bar(
    top_wheat_yearwise, 
    x="wheat_production", 
    y="year", 
    text="wheat_production",  # Show values on bars
    labels={"wheat_production": "Total Wheat Production", "year": "Year"},
    title="TOP 10 WHEAT PRODUCTION IN UP IN YEARWISE",
    color="year",  # Different colors for each state
    orientation="h",  # Horizontal bar chart
    color_discrete_sequence=px.colors.sequential.Viridis   # Use Blues_r color palette
)

# Show values on hover
fig.update_traces(texttemplate="%{text:.2s}", textposition="outside", hoverinfo="x+y")

# Show figure
fig.show()

In [ ]:
#9.Millet Production (Last 50y)
yearwise_MILLET_production = df.groupby("year")[["pearl_millet_production", "finger_millet_production"]].sum().reset_index()
millet_prod = yearwise_MILLET_production.tail(50)

# Create Interactive Line Plot
fig = px.line(
    millet_prod, 
    x="year", 
    y=["pearl_millet_production","finger_millet_production"] ,
    markers=True,  
    labels={"value": "Millet Production", "year": "Year", "variable": "Millet Type"},
    title="India's Millet Production Over the Last 50 Years",
    color_discrete_sequence=px.colors.sequential.Viridis   # Use Viridis color palette
)

# Improve hover details
fig.update_traces(mode="lines+markers", hoverinfo="x+y")
#"lines" → Draws a continuous line connecting data points."markers" → Adds distinct points (dots) at each data value for better visibility.
#"lines+markers" → Combines both, ensuring the trend is visible while making individual data points easy to spot.

# Show figure
fig.show()


In [ ]:
#10.Sorghum Production (Kharif and Rabi) by Region
region_Sorghum_production = df.groupby("state_name")[["kharif_sorghum_production", "rabi_sorghum_production"]].sum().reset_index()


# Reshape DataFrame for easier plotting
df_melted = region_Sorghum_production.melt(id_vars="state_name", 
                                           var_name="Season", 
                                           value_name="Production")

# Create Grouped Bar Chart
fig = px.bar(
    df_melted, 
    x="state_name", 
    y="Production", 
    color="Season", 
    barmode="group",
    labels={"Production": "Sorghum Production (Tonnes)", "state_name": "State"},
    title="Sorghum Production (Kharif & Rabi) by State",
    color_discrete_map={"kharif_sorghum_production": "green", "rabi_sorghum_production": "blue"}
)

# Show Plot
fig.show()

In [ ]:
#11.Top 7 States for Groundnut Production
state_groundNut_production = df.groupby("state_name")["groundnut_production"].sum().reset_index()
top_groundnut_states = state_groundNut_production.nlargest(7,"groundnut_production") 

# Create Interactive Bar Chart
fig = px.bar(
    top_groundnut_states, 
    x="groundnut_production", 
    y="state_name", 
    text="groundnut_production",  # Show values on bars
    labels={"groundnut_production": "Total GroundNut Production", "state_name": "State"},
    title="Top 7 States for Groundnut Production",
    color="state_name",  # Different colors for each state
    orientation="h",  # Horizontal bar chart
    color_discrete_sequence=px.colors.qualitative.Pastel1  
)

# Show values on hover
fig.update_traces(texttemplate="%{text:.2s}", textposition="outside", hoverinfo="x+y")

# Show figure
fig.show()

In [ ]:
#12.Soybean Production by Top 5 States and Yield Efficiency
soyabean_prod = df.groupby(["state_name","soyabean_yield"])["soyabean_production"].sum().reset_index()
top_soyabean_states = soyabean_prod.nlargest(5,"soyabean_production") 

# Scatter Plot for Yield Efficiency vs. Production
fig = px.scatter(
    top_soyabean_states, 
    x="soyabean_yield", 
    y="soyabean_production", 
    size="soyabean_production",  # Bubble size based on total production
    hover_name="state_name",
    labels={"soyabean_yield": "Yield Efficiency (kg/ha)", "soyabean_production": "Soybean Production (Tonnes)"},
    title="Soybean Production vs. Yield Efficiency (Top 5 States)",
    color="state_name"
)

fig.show()


In [ ]:
#13.Oilseed Production in Major States
oilSeed_prod = df.groupby("state_name")["oilseeds_production"].sum().reset_index()

fig = px.bar(
    oilSeed_prod, 
    x="state_name", 
    y="oilseeds_production", 
    color="state_name",
    text="oilseeds_production",
    labels={"oilseeds_production": "Oilseed Production (Tonnes)", "state_name": "State"},
    title="Oilseed Production in Major States",
)
fig.update_traces(texttemplate='%{text}', textposition='outside')  # Display production values
fig.show()

In [ ]:
#14. Impact of Area Cultivated on Production (Rice, Wheat, Maize)
# Creating a structured DataFrame
crop_data = pd.DataFrame({
    "Crop": ["Rice", "Wheat", "Maize"],
    "Crop Area (ha)": [df["rice_area"].sum(), df["wheat_area"].sum(), df["maize_area"].sum()],
    "Crop Production (tonnes)": [df["rice_production"].sum(), df["wheat_production"].sum(), df["maize_production"].sum()]
})

# Scatter Plot: Cultivated Area vs. Production
fig = px.scatter(
    crop_data, 
    x="Crop Area (ha)", 
    y="Crop Production (tonnes)", 
    color="Crop", 
    size="Crop Production (tonnes)",  # Bubble size based on total production
    hover_name="Crop",
    title="Impact of Area Cultivated on Production (Rice, Wheat, Maize)",
    labels={"Crop Area (ha)": "Cultivated Area (Hectares)", "Crop Production (tonnes)": "Total Production (Tonnes)"},
)

fig.show()


In [ ]:
# 15.Rice vs. Wheat Yield Across States

yield_data = df.groupby("state_name")[["rice_yield", "wheat_yield"]].mean().reset_index()

# Convert data to long format for bar chart
yield_data_melted = yield_data.melt(id_vars="state_name", var_name="Crop", value_name="Yield (kg/ha)")

# Bar Chart
fig = px.bar(
    yield_data_melted, 
    x="state_name", 
    y="Yield (kg/ha)", 
    color="Crop", 
    barmode="group",
    title="Rice vs. Wheat Yield Across States",
    labels={"Yield (kg/ha)": "Yield (kg/ha)", "state_name": "State"}
)

fig.show()

In [ ]:
print(df["state_name"].unique())

In [ ]:
!pip install mysql-connector-python

In [ ]:
import mysql.connector
mydb = mysql.connector.connect(
    host="localhost",
    port = 3310,
    user="root",
    password="Nivi20",
    database = "agricultural_Data",
)
mycursor = mydb.cursor(buffered=True) # cursor is like api to connect to server

In [ ]:
#creation of database
mycursor.execute("create database Agricultural_Data")

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# 2️⃣ Create an SQLAlchemy Engine -  it acts as a connection point between your application and the database.
engine = create_engine(f"mysql+mysqlconnector://{"root"}:{"Nivi20"}@{"localhost"}:{3310}/{"Agricultural_Data"}")

# 4️⃣ Move DataFrame into SQL Table
df.to_sql("Crops_Data", engine, if_exists="replace", index=False)

print("Data successfully inserted into MySQL!")

In [ ]:
!pip install tabulate

In [ ]:
mycursor.execute("SHOW DATABASES;")
for db in mycursor.fetchall():
    print(db)

In [10]:
# 1.Year-wise Trend of Rice Production Across States (Top 3)
mycursor.execute(
    """WITH Top_States AS (
    SELECT state_name 
    FROM Crops_Data
    GROUP BY state_name
    ORDER BY SUM(rice_production) DESC
    LIMIT 3
)
SELECT A.year, A.state_name, SUM(A.rice_production) AS rice_production
FROM Crops_Data A
JOIN Top_States B ON A.state_name = B.state_name
GROUP BY A.year, A.state_name
ORDER BY A.year, rice_production DESC;"""
)

out = mycursor.fetchall()

from tabulate import tabulate
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='psql'))


+--------+---------------+-------------------+
|   year | state_name    |   rice_production |
|--------+---------------+-------------------|
|   1966 | West Bengal   |       4.8194e+09  |
|   1966 | Uttar Pradesh |       2.4735e+09  |
|   1966 | Punjab        |       3.38e+08    |
|   1967 | West Bengal   |       4.86577e+09 |
|   1967 | Uttar Pradesh |       2.969e+09   |
|   1967 | Punjab        |       4.15e+08    |
|   1968 | West Bengal   |       5.46149e+09 |
|   1968 | Uttar Pradesh |       2.6071e+09  |
|   1968 | Punjab        |       4.695e+08   |
|   1969 | West Bengal   |       5.36282e+09 |
|   1969 | Uttar Pradesh |       3.0872e+09  |
|   1969 | Punjab        |       5.346e+08   |
|   1970 | West Bengal   |       5.20907e+09 |
|   1970 | Uttar Pradesh |       3.2656e+09  |
|   1970 | Punjab        |       6.85e+08    |
|   1971 | West Bengal   |       4.9651e+09  |
|   1971 | Uttar Pradesh |       3.5306e+09  |
|   1971 | Punjab        |       9.19e+08    |
|   1972 | We

In [ ]:
# 2.Top 5 Districts by Wheat Yield Increase Over the Last 5 Years


In [ ]:
import mysql.connector
mydb = mysql.connector.connect(
    host="localhost",
    port = 3310,
    user="root",
    password="Nivi20",
    database = "agricultural_Data",
)
mycursor = mydb.cursor(buffered=True) # cursor is like api to connect to server

In [ ]:

# 1.Year-wise Trend of Rice Production Across States (Top 3)
# 2.Top 5 Districts by Wheat Yield Increase Over the Last 5 Years
# 3.States with the Highest Growth in Oilseed Production (5-Year Growth Rate)
# 4.District-wise Correlation Between Area and Production for Major Crops (Rice, Wheat, and Maize)
# 5.Yearly Production Growth of Cotton in Top 5 Cotton Producing States
# 6.Districts with the Highest Groundnut Production in 2020
# 7.Annual Average Maize Yield Across All States
# 8.Total Area Cultivated for Oilseeds in Each State
# 9.Districts with the Highest Rice Yield
# 10.Compare the Production of Wheat and Rice for the Top 5 States Over 10 Years
